In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite


In [3]:
# Load the SM3 A. Nimodel
model = read_sbml_model('../model/40694_2018_60_MOESM3_ESM.xml')
model

'' is not a valid SBML 'SId'.


Name,
Memory address,0x07fd5717064c0
Number of metabolites,1818
Number of reactions,2320
Number of groups,0
Objective expression,1.0*DRAIN_Biomass - 1.0*DRAIN_Biomass_reverse_fcc30
Compartments,"Cytoplasm, Mitochondria, Extracellular, Peroxisome, Nucleus, Golgi, Vacuole"


In [ ]:
# Find pathway and metabolites

In [ ]:
# Add reactions
